In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from loguru import logger
from dotenv import load_dotenv
from pydantic import BaseModel
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.sparse as sparse
import torch.optim as optim

import numpy as np # required for the scikit-learn pipeline to work
import pandas as pd
import plotly.express as px
import mlflow

load_dotenv()

sys.path.insert(0, '..')

from src.viz import blueq_colors

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L5 - Reco Algo"
    run_name: str = '002-content-based'
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = 'user_id'
    item_col: str = 'parent_asin'
    rating_col: str = 'rating'
    
    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        
        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )
            import mlflow

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self
    
args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-13 19:26:04.498 | INFO     | __main__:init:28 - Setting up MLflow experiment FSDS RecSys - L5 - Reco Algo - run 002-content-based...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L5 - Reco Algo",
  "run_name": "002-content-based",
  "notebook_persist_dp": "/home/dvquys/frostmourne/reco-algo/notebooks/data/002-content-based",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Implement

In [4]:
from src.train_utils import mse_loss, train, MetricLogCallback
from src.model import ContentBased
import joblib

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# device = 'cpu'
logger.info(f"Using {device} device")

2024-09-13 19:26:04.885 | INFO     | __main__:<module>:9 - Using cuda device


In [6]:
item_metadata_pipeline = joblib.load('../data/item_metadata_pipeline.joblib')

# Test implementation

In [7]:
# Mock data
user_ids = [0, 0, 1, 2, 2]
item_ids = [0, 1, 2, 3, 4]
ratings = [1, 4, 5, 3, 2]
timestamps = [0, 1, 2, 3, 4]
main_category = ['All Electronics', 'Video Games', 'All Electronics', 'Video Games', "Unknown"]
title = ['All Electronics', 'Video Games', 'All Electronics', 'Video Games', "Unknown"]
description = [[], [], ["Video games blah blah"], [], ["blah blah"]]
categories = [[], ["Headsets"], ["Video Games"], [], ["blah blah"]]
price = ["from 14.99", "14.99", "price: 9.99", "20 dollars", "None"]

train_df = pd.DataFrame({
    "user_ids": user_ids,
    "item_ids": item_ids,
    "ratings": ratings,
    "timestamps": timestamps,
    "main_category": main_category,
    "title": title,
    "description": description,
    "categories": categories,
    "price": price,
})
# Drop duplicated item features so that the ContentBased model will fit correctly in terms of index mapping
fit_df = train_df.drop_duplicates(subset=['item_ids'])
train_item_features = item_metadata_pipeline.transform(fit_df).astype(np.float32)

val_user_ids = [0, 1, 2]
val_item_ids = [2, 1, 2]
val_ratings = [2, 4, 5]
val_timestamps = [5, 6, 7]
val_main_category = ['All Electronics', 'Video Games', 'All Electronics']
val_title = ['All Electronics', 'Video Games', 'All Electronics']
val_description = [["Video games blah blah"], [], ["Video games blah blah"]]
val_categories = [["Video Games"], ["Headsets"], ["Video Games"]]
val_price = ["price: 9.99", "14.99", "price: 9.99"]

val_df = pd.DataFrame({
    "user_ids": val_user_ids,
    "item_ids": val_item_ids,
    "ratings": val_ratings,
    "timestamps": val_timestamps,
    "main_category": val_main_category,
    "title": val_title,
    "description": val_description,
    "categories": val_categories,
    "price": val_price,
})
val_item_features = item_metadata_pipeline.transform(val_df).astype(np.float32)

In [8]:
n_users = len(set(user_ids))
n_items = len(set(item_ids))

print("Mock User IDs:", user_ids)
print("Mock Item IDs:", item_ids)
print("Ratings:", ratings)

model = ContentBased(item_features=train_item_features.todense(), device=device)

items1 = [1, 2]
items2 = [0, 3]
predictions = model.predict(items1, items2)
print(predictions)

print("\n\n")

users = [1]
users_last_item = 2
recommendations = model.recommend([users_last_item], k=args.top_K, progress_bar_type='tqdm_notebook')
print(recommendations)

Mock User IDs: [0, 0, 1, 2, 2]
Mock Item IDs: [0, 1, 2, 3, 4]
Ratings: [1, 4, 5, 3, 2]
tensor([7.5044e-05, 5.7788e-05], device='cuda:0')





Generating Recommendations:   0%|          | 0/1 [00:00<?, ?it/s]

{'item_indice': [2, 2, 2, 2], 'recommendation': [0, 3, 1, 4], 'score': [0.5773647427558899, 5.7788234698818997e-05, 4.33135537605267e-05, 0.0]}


# Prep data

In [9]:
train_df = pd.read_parquet("../data/train_item_features.parquet")
val_df = pd.read_parquet("../data/val_item_features.parquet")
val_timestamp = 1628643414042  # https://amazon-reviews-2023.github.io/data_processing/5core.html
full_df = pd.concat([train_df, val_df], axis=0)

In [10]:
user_col = 'user_id'
item_col = 'parent_asin'
timestamp_col = 'timestamp' 
rating_col = 'rating'

In [11]:
from src.id_mapper import IDMapper

In [12]:
user_ids = full_df['user_id'].values
item_ids = full_df['parent_asin'].values
unique_user_ids = list(set(user_ids))
unique_item_ids = list(set(item_ids))

logger.info(f"{len(unique_user_ids)=:,.0f}, {len(unique_item_ids)=:,.0f}")

2024-09-13 19:26:05.586 | INFO     | __main__:<module>:6 - len(unique_user_ids)=5,223, len(unique_item_ids)=2,653


In [13]:
idm = IDMapper()
idm.fit(unique_user_ids, unique_item_ids)

In [14]:
def map_indices(df):
    return (
        df
        .assign(
            user_index=df[user_col].apply(idm.get_user_index),
            item_index=df[item_col].apply(idm.get_item_index),
        )
    )

In [15]:
full_df = full_df.pipe(map_indices)
full_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price,user_index,item_index
0,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B0863MT183,4.0,1613701986538,All Electronics,KIWI design VR Stand Compatible with Quest 2/R...,[],"[Video Games, PC, Virtual Reality, Headsets]",19.99,3762,298
1,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B08P8P7686,5.0,1613702112995,All Electronics,Seltureone Accessories Compatible for Quest 2 ...,[],"[Video Games, PC, Virtual Reality, Headsets]",20.99,3762,2163
2,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B0B7LV3DN2,4.0,1617641445475,Cell Phones & Accessories,Eyglo Replacement Elite Strap for Oculus/Meta ...,[],"[Video Games, PC, Virtual Reality, Headsets]",28.49,3762,1632
3,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B09WMQ6DXG,5.0,1620231368468,All Electronics,"AMVR Controller Grips for Quest 2, Controller ...",[],"[Video Games, PC, Virtual Reality, Controllers]",15.99,3762,153
4,AHV6QCNBJNSGLATP56JAWJ3C4G2A,B019WRM1IA,5.0,1451860309000,All Electronics,Microsoft Xbox 360 Wired Controller for Window...,"[Product Description, Precisely what you need ...","[Video Games, Legacy Systems, Xbox Systems, Xb...",67.83,2991,1222
...,...,...,...,...,...,...,...,...,...,...,...
4254,AFTJ4C6AVWJSOPT3NKUX5JQJDUKA,B0BYNMZ3SP,5.0,1643164686173,Industrial & Scientific,Easy Hood USB C to USB C Cable Compatible for ...,[],"[Video Games, PC, Virtual Reality, Accessories]",9.99,4096,2089
4255,AFG6UJ2SWJJPMQXSW77MFJKHCEJQ,B08FC5L3RG,1.0,1634511131032,Video Games,PlayStation 5 Console,[The PS5 console unleashes new gaming possibil...,"[Video Games, PlayStation 5, Consoles]",448.19,1683,841
4256,AHAYPUV4RUPVQ2EYVVJBOFFKCFPA,B0CB9GDK9P,5.0,1638698211953,Video Games,PowerA MOGA Mobile Clip 2.0 for Xbox Controlle...,[Jump into mobile gaming with this expertly en...,"[Video Games, Legacy Systems, Xbox Systems, Xb...",14.99,2286,965
4257,AFVIZWLSRFUWN65MI4VT4JATJZIA,B07R9PBHP2,3.0,1656269943479,Video Games,Ace Combat 7: Skies Unknown - Xbox One,[Putting gamers in the cockpit of the most adv...,"[Video Games, Xbox One, Games]",19.99,1558,2385


# Recommend

In [16]:
from src.dataset_loader import ItemSequenceDataset
from torch.utils.data import DataLoader

In [17]:
fit_df = train_df.drop_duplicates(subset=[args.item_col])
train_item_features = item_metadata_pipeline.transform(fit_df).astype(np.float32)
model = ContentBased(item_features=train_item_features.todense(), device=device)

In [18]:
full_df

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price,user_index,item_index
0,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B0863MT183,4.0,1613701986538,All Electronics,KIWI design VR Stand Compatible with Quest 2/R...,[],"[Video Games, PC, Virtual Reality, Headsets]",19.99,3762,298
1,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B08P8P7686,5.0,1613702112995,All Electronics,Seltureone Accessories Compatible for Quest 2 ...,[],"[Video Games, PC, Virtual Reality, Headsets]",20.99,3762,2163
2,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B0B7LV3DN2,4.0,1617641445475,Cell Phones & Accessories,Eyglo Replacement Elite Strap for Oculus/Meta ...,[],"[Video Games, PC, Virtual Reality, Headsets]",28.49,3762,1632
3,AEVPPTMG43C6GWSR7I2UGRQN7WFQ,B09WMQ6DXG,5.0,1620231368468,All Electronics,"AMVR Controller Grips for Quest 2, Controller ...",[],"[Video Games, PC, Virtual Reality, Controllers]",15.99,3762,153
4,AHV6QCNBJNSGLATP56JAWJ3C4G2A,B019WRM1IA,5.0,1451860309000,All Electronics,Microsoft Xbox 360 Wired Controller for Window...,"[Product Description, Precisely what you need ...","[Video Games, Legacy Systems, Xbox Systems, Xb...",67.83,2991,1222
...,...,...,...,...,...,...,...,...,...,...,...
4254,AFTJ4C6AVWJSOPT3NKUX5JQJDUKA,B0BYNMZ3SP,5.0,1643164686173,Industrial & Scientific,Easy Hood USB C to USB C Cable Compatible for ...,[],"[Video Games, PC, Virtual Reality, Accessories]",9.99,4096,2089
4255,AFG6UJ2SWJJPMQXSW77MFJKHCEJQ,B08FC5L3RG,1.0,1634511131032,Video Games,PlayStation 5 Console,[The PS5 console unleashes new gaming possibil...,"[Video Games, PlayStation 5, Consoles]",448.19,1683,841
4256,AHAYPUV4RUPVQ2EYVVJBOFFKCFPA,B0CB9GDK9P,5.0,1638698211953,Video Games,PowerA MOGA Mobile Clip 2.0 for Xbox Controlle...,[Jump into mobile gaming with this expertly en...,"[Video Games, Legacy Systems, Xbox Systems, Xb...",14.99,2286,965
4257,AFVIZWLSRFUWN65MI4VT4JATJZIA,B07R9PBHP2,3.0,1656269943479,Video Games,Ace Combat 7: Skies Unknown - Xbox One,[Putting gamers in the cockpit of the most adv...,"[Video Games, Xbox One, Games]",19.99,1558,2385


In [19]:
batch_size = 128

dataset = ItemSequenceDataset(
    full_df,
    'user_index',
    'item_index',
    'rating',
    'timestamp',
    val_timestamp=val_timestamp,
    is_train=True
)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

val_dataset = ItemSequenceDataset(
    full_df,
    'user_index',
    'item_index',
    'rating',
    'timestamp',
    val_timestamp=val_timestamp,
    is_train=False
)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [20]:
full_df.loc[lambda df: df['user_id'].eq('AGS4TR4K5DMBRAFNBYSB2I2RCHHQ')]

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price,user_index,item_index
2849,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B07PB2DYTT,1.0,1556683035336,All Electronics,RGEEK FreeMcBoot FMCB 1.966 PS2 Memory Card 64...,[],"[Video Games, Legacy Systems, PlayStation Syst...",14.99,4755,657
0,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B0936HDGJ6,5.0,1652494657651,All Electronics,ISENPENK Switch Controller for Nintendo with W...,[],"[Video Games, Nintendo Switch, Accessories, Co...",21.99,4755,439
3875,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B07RN8JGZB,1.0,1632141542619,Computers,2PACK PS2 Controller Extension Cable Cord 6FT/...,[This extension cable is the perfect choice fo...,"[Video Games, Legacy Systems, PlayStation Syst...",8.99,4755,2578


In [21]:
def get_last_item(user_indice: int, timestamp: int):
    return val_dataset.get_item_sequence(user_indice, timestamp)[-1]

In [22]:
output = {
    "user_indice": [],
    "user_last_item_indice": [],
    "recommendation": [],
    "score": [],
}
for i, row in tqdm(val_df.iterrows(), total=val_df.shape[0]):
    user_indice = idm.get_user_index(row.user_id)
    last_item = get_last_item(user_indice, row.timestamp)
    recommendations = model.recommend([last_item], k=100, progress_bar_type=None)
    output['user_indice'].extend([user_indice] * len(recommendations['item_indice']))
    output['recommendation'].extend(recommendations['recommendation'])
    output['score'].extend(recommendations['score'])
    output["user_last_item_indice"].extend(recommendations['item_indice'])

  0%|          | 0/4259 [00:00<?, ?it/s]

# Evaluate

## Ranking metrics

In [23]:
user_col = args.user_col
item_col = args.item_col
rating_col = args.rating_col

In [24]:
from src.eval import create_label_df, create_rec_df, merge_recs_with_target

In [25]:
recommendations_df = pd.DataFrame(output).pipe(create_rec_df, idm)
recommendations_df

,user_indice,user_last_item_indice,recommendation,score,rec_ranking,user_id,parent_asin
0,4755,2578,1719,0.997569,1.0,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B0C3QWKTNQ
1,4755,2578,1729,0.745337,4.0,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B01FSK9KC0
2,4755,2578,1061,0.735777,6.0,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B07CWJC8ZM
3,4755,2578,2629,0.721426,7.0,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B01GYPB1HU
4,4755,2578,1443,0.700603,8.0,AGS4TR4K5DMBRAFNBYSB2I2RCHHQ,B06XH113GY
...,...,...,...,...,...,...,...
425895,4205,2566,1138,0.595772,96.0,AGT3DOVVP5JRBD3JNUK2OG23PMAA,B098JQ4WKW
425896,4205,2566,1678,0.592220,97.0,AGT3DOVVP5JRBD3JNUK2OG23PMAA,B017NFIGU0
425897,4205,2566,1054,0.590871,98.0,AGT3DOVVP5JRBD3JNUK2OG23PMAA,B00SZ1DQFM
425898,4205,2566,1508,0.590499,99.0,AGT3DOVVP5JRBD3JNUK2OG23PMAA,B0891JRTTJ


In [26]:
label_df = create_label_df(val_df)
label_df

,user_id,parent_asin,rating,rating_rank
3209,AEU4444ZVMLQB4ZXKLDCQL33BZPA,B07JH3LSHN,3.0,1.0
968,AF3LO27R5D3TCPUNE5U6ZQWMRHAA,B09LTYGYY2,5.0,1.0
2459,AGYCROALU32Q553KYALGCZLDPOSQ,B0039QJKZ8,5.0,1.0
1904,AEZ4WKT6DIOZ5ZC2KXIYU4PUXMDA,B08M2K9K67,5.0,1.0
1234,AF2AAA4CWRVF2IYVE7WB6OOIEMFA,B07SJVCKQW,5.0,1.0
...,...,...,...,...
1390,AGVZRX53LPVHDZQC7SEC7JMHOM3Q,B01MG8P418,4.0,10.0
247,AHZGMQN5OMOHJAJ6F2YPOS66RSDQ,B07MZ6PDG9,5.0,10.0
848,AEYWWBRMNORKNO6RKCNDN5D5ROTA,B07QKVKCT6,5.0,11.0
2436,AHZGMQN5OMOHJAJ6F2YPOS66RSDQ,B06ZY6VHDD,4.0,11.0


In [27]:
eval_df = merge_recs_with_target(recommendations_df, label_df, k=args.top_K)
eval_df

,user_indice,user_last_item_indice,recommendation,score,rec_ranking,user_id,parent_asin,rating,rating_rank
36,4565.0,1111.0,2181.0,0.856743,1,AE22LPCN47WUTHSG67R6SKN4A4MQ,B07CZXQS2V,0,NaN
95,4565.0,1111.0,2496.0,0.835941,2,AE22LPCN47WUTHSG67R6SKN4A4MQ,B0BJ92CHSC,0,NaN
55,4565.0,1111.0,2593.0,0.831148,3,AE22LPCN47WUTHSG67R6SKN4A4MQ,B082B4HBZ8,0,NaN
88,4565.0,1111.0,1116.0,0.829993,4,AE22LPCN47WUTHSG67R6SKN4A4MQ,B09GZP8V9W,0,NaN
12,4565.0,1111.0,1354.0,0.808340,5,AE22LPCN47WUTHSG67R6SKN4A4MQ,B006HZA55W,0,NaN
...,...,...,...,...,...,...,...,...,...
429914,1174.0,1253.0,2063.0,0.591575,97,AHZZ26USAR7T6VXZ7XIMHB7E3XEQ,B08T94NJ42,0,NaN
429876,1174.0,1253.0,1344.0,0.591557,98,AHZZ26USAR7T6VXZ7XIMHB7E3XEQ,B07P8PP67R,0,NaN
429924,1174.0,1253.0,1898.0,0.590177,99,AHZZ26USAR7T6VXZ7XIMHB7E3XEQ,B09VYZ4LLT,0,NaN
429905,1174.0,1253.0,152.0,0.589397,100,AHZZ26USAR7T6VXZ7XIMHB7E3XEQ,B08GGDT47N,0,NaN


### Visualize

In [28]:
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report
from evidently.metrics import PrecisionTopKMetric
from evidently.metrics import RecallTopKMetric
from evidently.metrics import FBetaTopKMetric
from evidently.metrics import NDCGKMetric
import warnings

warnings.filterwarnings(
    action='ignore',
    category=FutureWarning,
    module=r'evidently.metrics.recsys.precision_recall_k'
)

from src.viz import color_scheme

In [29]:
column_mapping = ColumnMapping(
    recommendations_type='rank',
    target=rating_col,
    prediction='rec_ranking',
    item_id=item_col,
    user_id=user_col
)

report = Report(metrics=[
    NDCGKMetric(k=args.top_k),
    RecallTopKMetric(k=args.top_K),
    PrecisionTopKMetric(k=args.top_k),
    FBetaTopKMetric(k=args.top_k),
], options=[color_scheme])

report.run(
    reference_data=None,
    current_data=eval_df,
    column_mapping=column_mapping
)

evidently_report_fp = f"{args.notebook_persist_dp}/evidently_report.html"
os.makedirs(args.notebook_persist_dp, exist_ok=True)
report.save_html(evidently_report_fp)

if args.log_to_mlflow:
    mlflow.log_artifact(evidently_report_fp)
    for metric_result in report.as_dict()['metrics']:
        metric = metric_result['metric']
        result = metric_result['result']['current'].to_dict()
        for kth, metric_value in result.items():
            mlflow.log_metric(f"val_{metric}_at_k_as_step", metric_value, step=kth)

# Predict

In [40]:
val_df.iloc[[2]]

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price
2,AFQAPWVESEJYTNZC23LDPQOH7QBA,B09GM4283G,5.0,1630119475785,Video Games,PlayStation PULSE 3D Wireless Headset – Midnig...,[Ignite your gaming nights with the ultra-slee...,"[Video Games, PlayStation 5, Accessories, Gami...",99.0


In [43]:
user_id = 'AFQAPWVESEJYTNZC23LDPQOH7QBA'
timestamp = 1630119475785
full_df.loc[lambda df: df['user_id'].eq(user_id)].sort_values('timestamp', ascending=False)

,user_id,parent_asin,rating,timestamp,main_category,title,description,categories,price,user_index,item_index
3998,AFQAPWVESEJYTNZC23LDPQOH7QBA,B09Y5PNFT6,5.0,1634411930426,Computers,Corsair Virtuoso RGB Wireless Gaming Headset -...,[The CORSAIR Virtuoso RGB Wireless delivers a ...,"[Video Games, PC, Accessories, Headsets]",150.67,4668,37
2,AFQAPWVESEJYTNZC23LDPQOH7QBA,B09GM4283G,5.0,1630119475785,Video Games,PlayStation PULSE 3D Wireless Headset – Midnig...,[Ignite your gaming nights with the ultra-slee...,"[Video Games, PlayStation 5, Accessories, Gami...",99.0,4668,2301
4846,AFQAPWVESEJYTNZC23LDPQOH7QBA,B07624RBWB,5.0,1622749151959,Video Games,Nintendo Switch Pro Controller,[],"[Video Games, Nintendo Switch, Accessories, Co...",69.0,4668,986
4845,AFQAPWVESEJYTNZC23LDPQOH7QBA,B08JHX17ZZ,5.0,1621461425122,Video Games,Super Mario 3D World + Bowser's Fury - Nintend...,"[Power up, team up, one up to save the Sprixie...","[Video Games, Nintendo Switch, Games]",49.1,4668,338
4844,AFQAPWVESEJYTNZC23LDPQOH7QBA,B0BZ8GZW38,5.0,1621461003075,Computers,Razer Huntsman Mini 60% Gaming Keyboard: Fast ...,[Dominate on a different scale with the Razer ...,"[Video Games, PC, Accessories, Gaming Keyboards]",97.1,4668,1106
4843,AFQAPWVESEJYTNZC23LDPQOH7QBA,B088MYMD21,5.0,1621460946569,Video Games,Nintendo Paper Mario: The Origami King - Switch,[A new paper-crafted Mario adventure unfolds o...,"[Video Games, Nintendo Switch, Accessories]",49.99,4668,2594
4842,AFQAPWVESEJYTNZC23LDPQOH7QBA,B0C89J78ZW,5.0,1601752199803,Computers,Logitech G Pro Wireless Gaming Mouse & G PRO M...,[Logitech G Pro Wireless Gaming Mouse with Esp...,"[Video Games, PC, Accessories, Gaming Mice]",176.93,4668,1742


In [44]:
user_indice = idm.get_user_index(user_id)
user_last_item_indice = get_last_item(user_indice, 1630119475785)
user_last_item_indice

np.int64(986)

In [45]:
item_id = 'B09GM4283G'
item_indice = idm.get_item_index(item_id)

model.predict([item_indice], [user_last_item_indice])

tensor([0.3102], device='cuda:0')

# Clean up

In [46]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/13 19:31:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run 002-content-based at: http://localhost:5003/#/experiments/1/runs/6b8d70d4e03645d2b1cc5be688969281.
2024/09/13 19:31:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/1.
